In [10]:
import cv2
import numpy as np
import csv
import os

def combine_distances(distance_geometry, distance_color):
    return distance_geometry + 0.5 * distance_color

def match_contours(contours_curr, contours_next, img_curr, img_next):
    matches = []
    for contour_curr in contours_curr:
        best_match = (None, None, float('inf'))
        for contour_next in contours_next:
            distance_geometry = cv2.matchShapes(contour_curr, contour_next, cv2.CONTOURS_MATCH_I1, 0.0)
            distance_color = cv2.compareHist(img_curr.astype(np.float32), img_next.astype(np.float32), cv2.HISTCMP_BHATTACHARYYA)
            distance_combined = combine_distances(distance_geometry, distance_color)
            if distance_combined < best_match[2]:
                best_match = (contour_curr, contour_next, distance_combined)
        matches.append(best_match)
    return matches

# Загрузка изображений для текущего дня
image_top = cv2.imread("1.12.top_986.jpg")
image_front = cv2.imread("1.12.front_986.jpg")

# Загрузка изображений для 21-го дня
image_top_day_21 = cv2.imread("16.12.top_986.jpg")
image_front_day_21 = cv2.imread("16.12.front_986.jpg")

# Проверка и приведение размеров изображений
min_height = min(image_top.shape[0], image_front.shape[0], image_top_day_21.shape[0], image_front_day_21.shape[0])
min_width = min(image_top.shape[1], image_front.shape[1], image_top_day_21.shape[1], image_front_day_21.shape[1])

image_top = image_top[:min_height, :min_width, :]
image_front = image_front[:min_height, :min_width, :]
image_top_day_21 = image_top_day_21[:min_height, :min_width, :]
image_front_day_21 = image_front_day_21[:min_height, :min_width, :]

# Преобразование изображений в цветовое пространство RGB
image_top_rgb = cv2.cvtColor(image_top, cv2.COLOR_BGR2RGB)
image_front_rgb = cv2.cvtColor(image_front, cv2.COLOR_BGR2RGB)
image_top_day_21_rgb = cv2.cvtColor(image_top_day_21, cv2.COLOR_BGR2RGB)
image_front_day_21_rgb = cv2.cvtColor(image_front_day_21, cv2.COLOR_BGR2RGB)

# Применение пороговой обработки для выделения зеленых пикселей на изображении сверху
lower_green = np.array([0, 50, 0], dtype=np.uint8)
upper_green = np.array([100, 255, 100], dtype=np.uint8)

mask_top = cv2.inRange(image_top_rgb, lower_green, upper_green)
mask_front = cv2.inRange(image_front_rgb, lower_green, upper_green)
mask_top_day_21 = cv2.inRange(image_top_day_21_rgb, lower_green, upper_green)
mask_front_day_21 = cv2.inRange(image_front_day_21_rgb, lower_green, upper_green)

# Применение операции морфологического закрытия для удаления шума
kernel = np.ones((10, 10), np.uint8)
closed_top = cv2.morphologyEx(mask_top, cv2.MORPH_CLOSE, kernel)
closed_front = cv2.morphologyEx(mask_front, cv2.MORPH_CLOSE, kernel)
closed_top_day_21 = cv2.morphologyEx(mask_top_day_21, cv2.MORPH_CLOSE, kernel)
closed_front_day_21 = cv2.morphologyEx(mask_front_day_21, cv2.MORPH_CLOSE, kernel)

# Поиск контуров зеленых регионов
contours_top, _ = cv2.findContours(closed_top, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours_front, _ = cv2.findContours(closed_front, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours_top_day_21, _ = cv2.findContours(closed_top_day_21, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours_front_day_21, _ = cv2.findContours(closed_front_day_21, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Создание тензоров для сохранения высот растков
heights_top = np.zeros(len(contours_top))
heights_front = np.zeros(len(contours_front))

# Измерение высот растков и сохранение в тензоры
for i, contour_top in enumerate(contours_top):
    height_top = cv2.contourArea(contour_top)
    heights_top[i] = height_top

for i, contour_front in enumerate(contours_front):
    height_front = cv2.contourArea(contour_front)
    heights_front[i] = height_front

# Вывод измеренных высот растков на первый день
print("\nВысоты растков на первый день (сверху):", heights_top)
print("Высоты растков на первый день (спереди):", heights_front)

# Сопоставление контуров между днями
contour_matches_top = match_contours(contours_top, contours_top_day_21, image_top_rgb, image_top_day_21_rgb)
matches_results_top = []

# Порог расстояния для считания контуров сопоставленными
matching_threshold = 0.1
# Порог высоты для считывания растений как выживших
scale_threshold = 0,3 
# Цвета для маркировки растений
marker_dead_color = (255, 0, 0)  
marker_alive_color = (142, 140, 83)  

# Вывод результатов сопоставления и формирование списка результатов
print("\nРезультаты сопоставления контуров (сверху):")
for match in contour_matches_top:
    contour_curr, contour_prev, distance = match
    is_matching = distance < matching_threshold and cv2.contourArea(contour_curr) > scale_threshold

    # Определение цвета маркера в зависимости от цвета растения
    if is_matching:
        # Проверка наличия зеленого цвета (RGB: 0, 255, 0)
        if np.any(image_top_rgb[contour_curr[:, 0, 1], contour_curr[:, 0, 0]] == [0, 255, 0]):
            marker_color = marker_alive_color  # Живые растения (зелёные)
        elif np.any(image_top_rgb[contour_curr[:, 0, 1], contour_curr[:, 0, 0]] == [255, 255, 0]) or \
                np.any(image_top_rgb[contour_curr[:, 0, 1], contour_curr[:, 0, 0]] == [139, 69, 19]):
            marker_color = marker_dead_color  # Мёртвые растения (жёлтые или коричневые)
        else:
            marker_color = (0, 0, 255) 
    else:
        marker_color = marker_dead_color

    # Получение ограничивающего прямоугольника и рисование маркера
    x, y, w, h = cv2.boundingRect(contour_curr)
    cv2.rectangle(image_top_rgb, (x, y), (x + w, y + h), marker_color, 2)

    matches_results_top.append(is_matching)
    print(f"Контур текущего дня: {contour_curr}, Контур предыдущего дня: {contour_prev}, Расстояние: {distance}, Совпадение: {is_matching}")

# Подсчет количества выживших растений на основе цветовых маркеров
survived_plants_color = sum(matches_results_top)
print(f"\nКоличество выживших растений на 21-й день (сверху) по цветовым маркерам: {survived_plants_color}")

# Проверка смерти растений по количеству выживших и порогу высоты
if survived_plants_color < len(contours_top) * scale_threshold:
    print("Растения находятся в состоянии умерших.")
else:
    print("Растения живы.")

# Вывод таблицы результатов
print("\nТаблица результатов (сверху):")
for i, is_match in enumerate(matches_results_top):
    print(f"Растение {i + 1}: {is_match}")

output_directory = "output_data"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
csv_file_path = os.path.join(output_directory, "результаты2.csv")

# Запись результатов в CSV файл
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Растение", "Совпадение"])
    for i, is_match in enumerate(matches_results_top):
        writer.writerow([f"Растение {i + 1}", is_match])

print(f"\nРезультаты сохранены в файл: {csv_file_path}")

# Вывод изображения с маркерами
cv2.imshow("Marked Image", cv2.cvtColor(image_top_rgb, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)
cv2.destroyAllWindows()



Высоты растков на первый день (сверху): [4.0000e+00 0.0000e+00 3.5043e+04 0.0000e+00]
Высоты растков на первый день (спереди): [    0.     0. 36565.     0.    63.]

Результаты сопоставления контуров (сверху):
Контур текущего дня: [[[230  45]]

 [[230  49]]

 [[231  49]]

 [[231  45]]], Контур предыдущего дня: [[[  0 171]]

 [[  0 172]]

 [[  5 172]]

 [[  4 171]]], Расстояние: 0.5934323744327158, Совпадение: False
Контур текущего дня: [[[42  7]]], Контур предыдущего дня: [[[69  0]]

 [[69  3]]], Расстояние: 0.12390347975522516, Совпадение: False
Контур текущего дня: [[[225   0]]

 [[225  14]]

 [[224  15]]

 [[212  15]]

 [[211  14]]

 [[211  11]]

 [[211  12]]

 [[209  14]]

 [[204  14]]

 [[202  16]]

 [[173  16]]

 [[172  15]]

 [[172   7]]

 [[172  15]]

 [[171  16]]

 [[156  16]]

 [[155  17]]

 [[146  17]]

 [[145  18]]

 [[136  18]]

 [[135  17]]

 [[135  16]]

 [[112  16]]

 [[111  15]]

 [[111  12]]

 [[106  12]]

 [[105  11]]

 [[105   7]]

 [[105  16]]

 [[104  17]]

 [[ 95